# Project Name

## Introduction

1. Introduction
2. Gathering data
3. Assessing data
4. Cleaning data
5. Storing, analyzing and visualizing wrangled data

In [31]:
import pandas as pd
import requests
import os
import tweepy
import json
import time

## Gathering

The WeRateDogs Twitter archive was made available for manual download from Udacity's Project Resources

In [ ]:
df = pd.read_csv('twitter-archive-enhanced.csv')

Hosted on Udacity's servers and download programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [2]:
# Download image-predictions.tsv file programmatically
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

response = requests.get(url)
with open(url.split('/')[-1], mode='wb') as file:
    file.write(response.content)

Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file.

In [3]:
# Create Tweepy API object
consumer_key = 'sghNXOl7D3URoUlpsOVoynzDg'
consumer_secret = 'ydjnx4Bvf9GzEyvxhBXxnEsQdQNP7VbS1oj4tv9u9Gf2MKrPuj'
access_token = '22273698-Ziz00jwPUaER1j6rQVlIQGEWQZmtEQEQWVkSkBZTi'
access_secret = 'vwH8LFHDLwzauhlhShVJcnN4PKTZDfB7wmhCrE5ENug9t'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [21]:
# Read tweet_ids from twitter-archive-enhanced.csv file into a list
tweet_id_list = df['tweet_id']

# Query the Twitter API for each tweet's JSON data using Tweepy library
#     and store each JSON data in the tweet_json.txt file
data = {}
id_errors = []

start = time.time()

with open('tweet_json.txt', 'w') as file:
    for tweet_id in tweet_id_list:
        try:
            data = api.get_status(tweet_id, text_mode='extended')
            file.write(json.dumps(data._json) + '\n')
        except Exception as e:
            id_errors.append(print('No status found with ID {}'.format(tweet_id)))

end = time.time()
print(end - start)

No status found with ID 888202515573088257
No status found with ID 873697596434513921
No status found with ID 872668790621863937
No status found with ID 872261713294495745
No status found with ID 869988702071779329
No status found with ID 866816280283807744
No status found with ID 861769973181624320
No status found with ID 856602993587888130
No status found with ID 851953902622658560
No status found with ID 845459076796616705
No status found with ID 844704788403113984
No status found with ID 842892208864923648
No status found with ID 837366284874571778
No status found with ID 837012587749474308
No status found with ID 829374341691346946
No status found with ID 827228250799742977


Rate limit reached. Sleeping for: 514


No status found with ID 812747805718642688
No status found with ID 802247111496568832
No status found with ID 779123168116150273
No status found with ID 775096608509886464
No status found with ID 771004394259247104
No status found with ID 770743923962707968
No status found with ID 759566828574212096
No status found with ID 754011816964026368
No status found with ID 680055455951884288


Rate limit reached. Sleeping for: 638


5879.627026081085


In [25]:
# Reat the tweet_json.txt file into a pandas DataFrame 
json_list = []

with open('tweet_json.txt') as json_file:
    for line in json_file:
        data = json.loads(line)
        tweet_id = data['id']
        favorite_counts = data['favorite_count']
        retweet_counts = data['retweet_count']
        
        json_list.append({'tweet_id': tweet_id,
                          'favorite_counts': favorite_counts,
                          'retweet_counts': retweet_counts})
        
df_tweet = pd.DataFrame(json_list, columns=['tweet_id', 'favorite_counts', 'retweet_counts'])

In [28]:
df_tweet.head()

,tweet_id,favorite_counts,retweet_counts
0,892420643555336193,36458,7755
1,892177421306343426,31424,5747
2,891815181378084864,23664,3801
3,891689557279858688,39775,7912
4,891327558926688256,37992,8547


## Assessing